# Face Recognition Based Attendance System

## Creating the model

In [ ]:
from math import sqrt
from sklearn import neighbors
from os import listdir
from os.path import isdir, join, isfile, splitext
import pickle
from PIL import Image, ImageFont, ImageDraw, ImageEnhance
import face_recognition
from face_recognition import face_locations
from face_recognition.cli import image_files_in_folder

ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

def train(train_dir, model_save_path = "knn_model.sav", n_neighbors = None, knn_algo = 'ball_tree', verbose=False):
    """
    Trains a k-nearest neighbors classifier for face recognition.
    :param train_dir: directory that contains a sub-directory for each known person, with its name.
     (View in source code to see train_dir example tree structure)
     Structure:
        <train_dir>/
        ├── <person1>/
        │   ├── <somename1>.jpeg
        │   ├── <somename2>.jpeg
        │   ├── ...
        ├── <person2>/
        │   ├── <somename1>.jpeg
        │   └── <somename2>.jpeg
        └── ...
    :param model_save_path: (optional) path to save model of disk
    :param n_neighbors: (optional) number of neighbors to weigh in classification. Chosen automatically if not specified.
    :param knn_algo: (optional) underlying data structure to support knn.default is ball_tree
    :param verbose: verbosity of training
    :return: returns knn classifier that was trained on the given data.
    """
    X = []
    y = []
    for class_dir in listdir(train_dir):
        if not isdir(join(train_dir, class_dir)):
            continue
        for img_path in image_files_in_folder(join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            faces_bboxes = face_locations(image)
            print(faces_bboxes)
            if len(faces_bboxes) != 1:
                if verbose:
                    print("image {} not fit for training: {}".format(img_path, "didn't find a face" if len(faces_bboxes) < 1 else "found more than one face"))
                continue
            X.append(face_recognition.face_encodings(image, known_face_locations=faces_bboxes)[0])
            y.append(class_dir)


    if n_neighbors is None:
        n_neighbors = int(round(sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically as:", n_neighbors)

    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    if model_save_path != "":
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)
    return knn_clf

def predict(X_img_path, knn_clf = None, model_save_path ="knn_model.sav", DIST_THRESH = .5):
    """
    recognizes faces in given image, based on a trained knn classifier
    :param X_img_path: path to image to be recognized
    :param knn_clf: (optional) a knn classifier object. if not specified, model_save_path must be specified.
    :param model_save_path: (optional) path to a pickled knn classifier. if not specified, model_save_path must be knn_clf.
    :param DIST_THRESH: (optional) distance threshold in knn classification. the larger it is, the more chance of misclassifying an unknown person to a known one.
    :return: a list of names and face locations for the recognized faces in the image: [(name, bounding box), ...].
        For faces of unrecognized persons, the name 'N/A' will be passed.
    """

    if not isfile(X_img_path) or splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("invalid image path: {}".format(X_img_path))

    if knn_clf is None and model_save_path == "knn_model.sav":
        raise Exception("must supply knn classifier either thourgh knn_clf or model_save_path")

    if knn_clf is None:
        with open(model_save_path, 'rb') as f:
            knn_clf = pickle.load(f)

    X_img = face_recognition.load_image_file(X_img_path)
    X_faces_loc = face_locations(X_img)
    if len(X_faces_loc) == 0:
        return []

    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_faces_loc)


    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)

    is_recognized = [closest_distances[0][i][0] <= DIST_THRESH for i in range(len(X_faces_loc))]
    #print(knn_clf.predict(faces_encodings))
    # predict classes and cull classifications that are not with high confidence
    return [(pred, loc) if rec else ("N/A", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_faces_loc, is_recognized)]

def draw_preds(img_path, preds):
    """
    shows the face recognition results visually.
    :param img_path: path to image to be recognized
    :param preds: results of the predict function
    :return:
    """
    source_img = Image.open(img_path).convert("RGBA")
    draw = ImageDraw.Draw(source_img)
    for pred in preds:
        loc = pred[1]
        name = pred[0]
        # (top, right, bottom, left) => (left,top,right,bottom)
        draw.rectangle(((loc[3], loc[0]), (loc[1],loc[2])), outline="red")
        draw.text((loc[3], loc[0] - 30), name, font=ImageFont.truetype('arial.ttf', 30),fill=(255,255,255,0))
    source_img.show()

if __name__ == "__main__":
    knn_clf = train("knn_examples/train")
    #knn_clf = pickle.load(open("knn_model.sav", 'rb'))
    

    
    for img_path in listdir("knn_examples/test"):
        preds = predict(join("knn_examples/test", img_path) ,knn_clf=knn_clf)
        print(preds)
        draw_preds(join("knn_examples/test", img_path), preds)


## Using the model by using webcam

In [2]:
import face_recognition
import cv2
import pickle
from sklearn import neighbors
import time

# This is a demo of running face recognition on live video from your webcam. It's a little more complicated than the
# other example, but it includes some basic performance tweaks to make things run a lot faster:
#   1. Process each video frame at 1/4 resolution (though still display it at full resolution)
#   2. Only detect faces in every other frame of video.

# PLEASE NOTE: This example requires OpenCV (the `cv2` library) to be installed only to read from your webcam.
# OpenCV is *not* required to use the face_recognition library. It's only required if you want to run this
# specific demo. If you have trouble installing it, try any of the other demos that don't require it instead.

# Get a reference to webcam #0 (the default one)    
    


video_capture = cv2.VideoCapture(0)


# Initialize some variables
face_locations = []
face_encodings = []
face_names = []
present=[]
c=[]
process_this_frame = True
if __name__ == "__main__":
    #knn_clf = train("knn_examples/train")
    knn_clf = pickle.load(open("knn_model.sav", 'rb'))
    
    while True:
        # Grab a single frame of video
        ret, frame = video_capture.read()

        # Resize frame of video to 1/4 size for faster face recognition processing
        small_frame = cv2.resize(frame, (0, 0), fx=0.25, fy=0.25)
        rgb_small_frame = small_frame[:, :, ::-1]
        
        
        

       
        

        # Only process every other frame of video to save time
        if process_this_frame:
            # Find all the faces and face encodings in the current frame of video
            face_locations = face_recognition.face_locations(rgb_small_frame)
            face_encodings = face_recognition.face_encodings(rgb_small_frame, face_locations)
            if (len(face_encodings)>0):
                
                closest_distances = knn_clf.kneighbors(face_encodings, n_neighbors=1)


                is_recognized = [closest_distances[0][i][0] <= 0.5 for i in range(len(face_locations))]

                face_names = []
                for pred, loc, rec in zip(knn_clf.predict(face_encodings),face_locations, is_recognized):
                     if rec:
                         face_names.append(pred)
                         if pred not in present: 
                                present.append(pred)
                                c.append(time.strftime('%H:%M:%S', time.localtime()))
                         
                         
                        
                     else :
                         face_names.append("unknown")
                 
        

        process_this_frame = not process_this_frame


        # Display the results
        for (top, right, bottom, left), name in zip(face_locations, face_names):
            # Scale back up face locations since the frame we detected in was scaled to 1/4 size
            top *= 4
            right *= 4
            bottom *= 4
            left *= 4

            # Draw a box around the face
            cv2.rectangle(frame, (left, top), (right, bottom), (0, 0, 255), 2)

            # Draw a label with a name below the face
            cv2.rectangle(frame, (left, bottom - 35), (right, bottom), (0, 0, 255), cv2.FILLED)
            font = cv2.FONT_HERSHEY_DUPLEX
            #print(name)
            cv2.putText(frame, name, (left + 6, bottom - 6), font, 1.0, (255, 255, 255), 1)


          



    

        # Display the resulting image
        cv2.imshow('Video', frame)

        # Hit 'q' on the keyboard to quit!
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break





        
# Release handle to the webcam
video_capture.release()
cv2.destroyAllWindows()


import pandas as pd

t=[0,0,0,0,0,0,0,0,0,0]
R=['101','102','103','104','105','106','107','108','109','110']
n=[ 'Aditya','Aefaaz','Amogh','Ashutosh','Harsh','Karan', 'Nitish', 'Sarosh','Shivam','Shreyas' ]
a=[0,0,0,0,0,0,0,0,0,0]
for i in range(len(present)):
    if present[i] in n: 
        a.pop(n.index(present[i]))
        a.insert(n.index(present[i]),'P')
        t.pop(n.index(present[i]))
        t.insert(n.index(present[i]),c[i])
for _ in range(len(a)):
        if a[_]==0:
            a.pop(_)
            a.insert(_,"-")
            t.pop(_)
            t.insert(_,"-")
data = {'Roll No.':R,'Name':n,'P/A':a,'Time':t}

df = pd.DataFrame(data)
df=df[['Roll No.','Name','P/A','Time']]

from IPython.display import HTML

def hover(hover_color="#ffff99"):
    return dict(selector="tr:hover",
                props=[("background-color", "%s" % hover_color)])

styles = [
    hover(),
    dict(selector="th", props=[("font-size", "150%"),
                               ("text-align", "center")]),
    dict(selector="caption", props=[("caption-side", "center")])
]
def highlight(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    z=[]
    for i in range(len(df.index)):
         if df.iloc[i,2]=='P':
                z.append(i)
         else:
                z.append(-1)     
    
    return ['background-color: greenyellow' if i==z[i] else ''for i in range(len(z))]
html = (df.style.set_table_styles(styles)
          .set_caption("Attendence Sheet.").set_properties(**{'font-size':'11pt'})
           .apply(highlight))

html

,Roll No.,Name,P/A,Time
0,101,Aditya,-,-
1,102,Aefaaz,-,-
2,103,Amogh,-,-
3,104,Ashutosh,-,-
4,105,Harsh,-,-
5,106,Karan,P,19:21:31
6,107,Nitish,-,-
7,108,Sarosh,-,-
8,109,Shivam,-,-
9,110,Shreyas,-,-
